# Chapter 16: mypy in Practice

This notebook covers practical usage of mypy - Python's most widely-used static
type checker. You will learn how to configure it, interpret its errors, add types
to existing code gradually, and use advanced features like type narrowing and casts.

## Key Concepts
- mypy configuration in `pyproject.toml`
- Common mypy errors and how to fix them
- Gradual typing: adding types to untyped code
- Type narrowing with `isinstance`, `assert`, and type guards
- `cast()` for when you know better than the checker
- `reveal_type()` for debugging type inference
- Best practices: when to use `Any`, when to be strict

## mypy Configuration in pyproject.toml

mypy is configured in `pyproject.toml` under the `[tool.mypy]` section.
Good configuration catches real bugs while avoiding excessive noise.
Below is a recommended starting configuration and an explanation of each option.

In [ ]:
# This cell shows a recommended pyproject.toml configuration.
# It is presented as a string since TOML files cannot be executed as Python.

mypy_config = """
[tool.mypy]
python_version = "3.12"           # Target Python version for type checking
warn_return_any = true             # Warn when a function returns Any
warn_unused_configs = true         # Warn about unused [mypy-*] sections
warn_redundant_casts = true        # Warn about unnecessary cast() calls
warn_unused_ignores = true         # Warn about unnecessary # type: ignore
check_untyped_defs = true          # Type-check inside untyped functions too
disallow_untyped_defs = false      # Don't require all functions to have types (gradual)
disallow_any_generics = false      # Allow bare list, dict (vs list[int], dict[str, int])
no_implicit_optional = true        # Don't auto-convert None defaults to Optional
strict_equality = true             # Flag nonsensical equality checks

# Per-module overrides: stricter for your code, lenient for third-party
[[tool.mypy.overrides]]
module = "my_project.*"            # Your project's modules
disallow_untyped_defs = true       # Require all functions to have type hints
warn_return_any = true

[[tool.mypy.overrides]]
module = "tests.*"                 # Test modules can be less strict
disallow_untyped_defs = false

[[tool.mypy.overrides]]
module = "third_party_lib.*"       # Ignore untyped third-party libraries
ignore_missing_imports = true
"""

print(mypy_config)

# Strictness progression (from loose to strict):
strictness_levels = [
    ("Level 1 - Minimal", "check_untyped_defs = true"),
    ("Level 2 - Moderate", "+ warn_return_any, no_implicit_optional"),
    ("Level 3 - Recommended", "+ disallow_untyped_defs (for your code)"),
    ("Level 4 - Strict", "--strict flag (all checks enabled)"),
]

print("Recommended strictness progression:")
for level, desc in strictness_levels:
    print(f"  {level}: {desc}")

## Common mypy Errors and How to Fix Them

mypy produces specific error codes that tell you exactly what went wrong.
Let's walk through the most common errors you will encounter and the
correct way to fix each one.

In [ ]:
from typing import Optional

# -------------------------------------------------------
# Error 1: Incompatible types in assignment [assignment]
# -------------------------------------------------------
# BAD: mypy error - Incompatible types (got "str", expected "int")
# x: int = "hello"

# FIX: Use the correct type
x: int = 42
y: str = "hello"
print(f"x={x}, y={y}")

# -------------------------------------------------------
# Error 2: Item "None" has no attribute [union-attr]
# -------------------------------------------------------
# BAD: mypy error - "Optional[str]" has no attribute "upper"
def get_name() -> str | None:
    return "Alice"

# name = get_name()
# print(name.upper())  # Error! name might be None

# FIX: Check for None first (type narrowing)
name = get_name()
if name is not None:
    print(f"Upper: {name.upper()}")  # mypy knows name is str here

# -------------------------------------------------------
# Error 3: Argument type incompatible [arg-type]
# -------------------------------------------------------
def double(n: int) -> int:
    return n * 2

# BAD: double("3")  # Error: expected int, got str

# FIX: Pass the correct type
print(f"double(3) = {double(3)}")

# -------------------------------------------------------
# Error 4: Missing return statement [return]
# -------------------------------------------------------
# BAD: not all paths return a value
# def categorize(n: int) -> str:
#     if n > 0:
#         return "positive"
#     elif n < 0:
#         return "negative"
#     # Missing: what about n == 0?

# FIX: Handle all cases
def categorize(n: int) -> str:
    if n > 0:
        return "positive"
    elif n < 0:
        return "negative"
    return "zero"  # All paths covered

print(f"categorize(5) = {categorize(5)}")
print(f"categorize(0) = {categorize(0)}")

# -------------------------------------------------------
# Error 5: Incompatible return type [return-value]
# -------------------------------------------------------
# BAD: returning wrong type
# def get_count() -> int:
#     return "three"  # Error: expected int, got str

# FIX: Return the declared type
def get_count() -> int:
    return 3

print(f"get_count() = {get_count()}")

## Gradual Typing: Adding Types to Untyped Code

You do not need to annotate everything at once. mypy supports **gradual typing** -
start with the most important interfaces and expand outward. The `Any` type acts
as an escape hatch: it is compatible with every other type.

In [ ]:
from typing import Any

# -------------------------------------------------------
# Step 1: Start with NO type hints (fully untyped)
# -------------------------------------------------------
def process_data_v1(data, config):
    """Version 1: No types. mypy skips checking this by default."""
    result = []
    for item in data:
        if config.get("uppercase"):
            result.append(item.upper())
        else:
            result.append(item)
    return result


# -------------------------------------------------------
# Step 2: Add types to public interfaces first
# -------------------------------------------------------
def process_data_v2(data: list[str], config: dict[str, Any]) -> list[str]:
    """Version 2: Public interface typed. Internal logic uses Any for config values."""
    result: list[str] = []
    for item in data:
        if config.get("uppercase"):
            result.append(item.upper())
        else:
            result.append(item)
    return result


# -------------------------------------------------------
# Step 3: Replace Any with precise types
# -------------------------------------------------------
from typing import TypedDict


class ProcessConfig(TypedDict, total=False):
    """Typed configuration for process_data."""
    uppercase: bool
    prefix: str
    max_length: int


def process_data_v3(data: list[str], config: ProcessConfig) -> list[str]:
    """Version 3: Fully typed, including configuration shape."""
    result: list[str] = []
    for item in data:
        processed = item
        if config.get("uppercase"):
            processed = processed.upper()
        prefix = config.get("prefix", "")
        if prefix:
            processed = f"{prefix}{processed}"
        max_len = config.get("max_length")
        if max_len is not None:
            processed = processed[:max_len]
        result.append(processed)
    return result


# All three versions work at runtime
data = ["hello", "world", "python"]

print("v1 (untyped):", process_data_v1(data, {"uppercase": True}))
print("v2 (partial):", process_data_v2(data, {"uppercase": True}))

config: ProcessConfig = {"uppercase": True, "prefix": ">> ", "max_length": 10}
print("v3 (fully typed):", process_data_v3(data, config))

## Type Narrowing with isinstance, assert, and Type Guards

**Type narrowing** is when the type checker recognizes that a variable's type
has been refined within a code block. mypy understands several patterns:
- `isinstance()` checks
- `is None` / `is not None` checks
- `assert` statements
- Custom `TypeGuard` functions (PEP 647)

In [ ]:
from typing import TypeGuard


# -------------------------------------------------------
# isinstance narrowing
# -------------------------------------------------------
def describe(value: int | str | list[int]) -> str:
    """mypy narrows the type in each branch."""
    if isinstance(value, int):
        # mypy knows: value is int
        return f"Integer: {value * 2}"
    elif isinstance(value, str):
        # mypy knows: value is str
        return f"String: {value.upper()}"
    else:
        # mypy knows: value is list[int]
        return f"List sum: {sum(value)}"


print(describe(42))
print(describe("hello"))
print(describe([1, 2, 3]))


# -------------------------------------------------------
# None narrowing (is / is not)
# -------------------------------------------------------
def process_optional(value: str | None) -> str:
    """Narrow Optional to remove None."""
    if value is None:
        return "<empty>"
    # mypy knows: value is str (not None)
    return value.strip().title()


print(f"\nprocess_optional(None) = {process_optional(None)!r}")
print(f"process_optional('  hello  ') = {process_optional('  hello  ')!r}")


# -------------------------------------------------------
# assert narrowing
# -------------------------------------------------------
def get_first_word(text: str | None) -> str:
    """Assert narrows None away."""
    assert text is not None, "text must not be None"
    # mypy knows: text is str after the assert
    return text.split()[0]


print(f"get_first_word('hello world') = {get_first_word('hello world')!r}")


# -------------------------------------------------------
# TypeGuard for custom narrowing (PEP 647)
# -------------------------------------------------------
def is_string_list(values: list[object]) -> TypeGuard[list[str]]:
    """Return True if all elements are strings.

    When this returns True, mypy narrows list[object] to list[str].
    """
    return all(isinstance(v, str) for v in values)


def process_items(items: list[object]) -> str:
    """Process items, with special handling for all-string lists."""
    if is_string_list(items):
        # mypy knows: items is list[str]
        return ", ".join(items)  # .join() works because items is list[str]
    return str(items)


print(f"\nprocess_items(['a', 'b', 'c']) = {process_items(['a', 'b', 'c'])!r}")
print(f"process_items([1, 2, 3]) = {process_items([1, 2, 3])!r}")


# -------------------------------------------------------
# Exhaustiveness checking with assert_never
# -------------------------------------------------------
from typing import Literal, assert_never

Status = Literal["active", "inactive", "pending"]


def handle_status(status: Status) -> str:
    """Handle all possible status values. mypy verifies exhaustiveness."""
    if status == "active":
        return "User is active"
    elif status == "inactive":
        return "User is inactive"
    elif status == "pending":
        return "User is pending approval"
    else:
        # If we add a new Status value but forget to handle it,
        # mypy will report an error here because status won't be Never
        assert_never(status)


print(f"\nhandle_status('active') = {handle_status('active')!r}")
print(f"handle_status('pending') = {handle_status('pending')!r}")

## cast() - When You Know Better Than the Checker

`cast(T, value)` tells mypy to treat `value` as type `T`. It performs **no runtime
check** - it is purely a type checker directive. Use it sparingly: every `cast` is
a promise you are making that mypy cannot verify.

In [ ]:
from typing import cast, Any
import json


# Scenario 1: JSON parsing returns Any, but you know the shape
raw_json = '{"name": "Alice", "age": 30}'
parsed: Any = json.loads(raw_json)  # json.loads returns Any

# You KNOW it is a dict with string keys
user_data = cast(dict[str, Any], parsed)
print(f"Name: {user_data['name']}")
print(f"Age: {user_data['age']}")


# Scenario 2: Container with known element types
mixed_storage: dict[str, object] = {
    "count": 42,
    "name": "Widget",
    "prices": [9.99, 19.99, 29.99],
}

# You know "count" is an int, but mypy sees object
count = cast(int, mixed_storage["count"])
doubled: int = count * 2  # Works because mypy thinks count is int
print(f"\nDoubled count: {doubled}")

# You know "prices" is a list of floats
prices = cast(list[float], mixed_storage["prices"])
total: float = sum(prices)
print(f"Total price: ${total:.2f}")


# Scenario 3: After a validation check that mypy cannot follow
def parse_config(raw: dict[str, Any]) -> tuple[str, int]:
    """Parse and validate config, then cast."""
    # Validation logic
    assert isinstance(raw["host"], str), "host must be a string"
    assert isinstance(raw["port"], int), "port must be an integer"

    # After validation, we know the types are correct
    host = cast(str, raw["host"])
    port = cast(int, raw["port"])
    return host, port


host, port = parse_config({"host": "localhost", "port": 8080})
print(f"\nConfig: {host}:{port}")


# WARNING: cast() does NOT check at runtime - it can hide bugs!
bad_cast = cast(int, "not an int")  # No error raised!
print(f"\nDangerous cast (no runtime error): {bad_cast!r}")
print(f"Type at runtime: {type(bad_cast).__name__}")  # Still str!

## reveal_type() for Debugging Type Inference

`reveal_type(expr)` is a special function recognized by mypy. When mypy analyzes
your code, it prints the inferred type of the expression. This is invaluable for
understanding what mypy "thinks" a variable's type is.

**Note**: `reveal_type()` is a mypy directive, not a runtime function. In Python
3.11+, `typing.reveal_type()` was added as a real function that also works at runtime.

In [ ]:
# In Python 3.11+, reveal_type is available from typing
from typing import reveal_type

# Basic inference
x = 42
reveal_type(x)  # mypy output: int  |  runtime: prints the type and value

y = [1, 2, 3]
reveal_type(y)  # mypy: list[int]

z = {"a": 1, "b": 2}
reveal_type(z)  # mypy: dict[str, int]


# Useful for understanding narrowing
def demo_narrowing(value: str | int | None) -> str:
    # Before any checks:
    # reveal_type(value)  # mypy: str | int | None

    if value is None:
        return "nothing"

    # After None check:
    reveal_type(value)  # mypy: str | int  (None eliminated)

    if isinstance(value, str):
        # After isinstance:
        reveal_type(value)  # mypy: str
        return value.upper()

    # After both checks:
    reveal_type(value)  # mypy: int (only possibility left)
    return str(value)


print(f"\ndemo_narrowing('hello') = {demo_narrowing('hello')}")
print(f"demo_narrowing(42) = {demo_narrowing(42)}")
print(f"demo_narrowing(None) = {demo_narrowing(None)}")


# Reveal type in complex expressions
data: dict[str, list[int]] = {"scores": [90, 85, 92]}
scores = data.get("scores", [])
reveal_type(scores)  # mypy: list[int]

first = scores[0] if scores else None
reveal_type(first)  # mypy: int | None

## The `# type: ignore` Comment

When you need to suppress a mypy error on a specific line, use `# type: ignore`.
Always include the error code to document **why** you are suppressing it and
to avoid accidentally hiding other errors.

In [ ]:
# BAD: bare type: ignore hides ALL errors on the line
# result = some_untyped_function()  # type: ignore

# GOOD: specific error code documents the intent
# result = some_untyped_function()  # type: ignore[no-untyped-call]

# Common error codes you might suppress:
error_codes = {
    "assignment": "Incompatible types in assignment",
    "arg-type": "Incompatible argument type",
    "return-value": "Incompatible return value type",
    "union-attr": "Attribute access on union type",
    "no-untyped-call": "Calling an untyped function",
    "no-untyped-def": "Function is missing type annotations",
    "import-untyped": "Importing an untyped module",
    "override": "Incompatible override of base class method",
    "misc": "Miscellaneous error",
}

print("Common mypy error codes:")
for code, description in error_codes.items():
    print(f"  [{code}] {description}")


# Practical example: known third-party library quirk
import json

# json.loads returns Any, but we validate the shape ourselves
raw = json.loads('{"value": 42}')

# Instead of cast, sometimes a targeted ignore is clearer:
value: int = raw["value"]  # type: ignore[assignment]  # validated by schema
print(f"\nExtracted value: {value}")

# When to use # type: ignore vs cast():
# - cast(): You want to assert a type and keep type checking downstream
# - type: ignore: You want to silence a specific false positive
print("\nRule of thumb:")
print("  cast() -> 'I know this type, continue checking from here'")
print("  type: ignore -> 'This line is a known false positive, skip it'")

## Best Practices: When to Use Any, When to Be Strict

Type annotations are a tool for catching bugs and communicating intent.
Being too strict wastes time; being too loose misses bugs. Here are
practical guidelines for finding the right balance.

In [ ]:
from typing import Any
from collections.abc import Callable


# -------------------------------------------------------
# GOOD uses of Any: genuine dynamism
# -------------------------------------------------------

# 1. Plugin systems where you truly accept anything
def register_plugin(name: str, plugin: Any) -> None:
    """Register a plugin. Plugins can be any object with any interface."""
    print(f"Registered plugin: {name} ({type(plugin).__name__})")


# 2. Logging / debugging utilities
def debug_log(label: str, value: Any) -> None:
    """Log any value for debugging."""
    print(f"[DEBUG] {label}: {value!r} (type={type(value).__name__})")


# 3. Wrapping truly dynamic APIs (e.g., JSON, pickle)
def load_json(path: str) -> Any:
    """Load JSON data. Shape is unknown until runtime."""
    import json
    # In real code, you'd validate the shape after loading
    return json.loads('{"key": "value"}')


register_plugin("my_plugin", {"handler": lambda: None})
debug_log("config", {"port": 8080, "debug": True})
print()


# -------------------------------------------------------
# BAD uses of Any: laziness or avoidance
# -------------------------------------------------------

# BAD: Using Any because you didn't think about the type
# def process(data: Any) -> Any:  # What goes in? What comes out? Who knows!
#     return data.do_thing()

# GOOD: Be specific about what you accept and return
from typing import Protocol


class Processable(Protocol):
    def do_thing(self) -> str: ...


def process(data: Processable) -> str:
    """Now both input and output types are clear."""
    return data.do_thing()


# -------------------------------------------------------
# Priority order for adding types
# -------------------------------------------------------
priorities = [
    ("1. Public API functions", "These are consumed by other code - types serve as docs"),
    ("2. Data models / classes", "__init__ parameters and key attributes"),
    ("3. Return types", "Helps callers understand what they get back"),
    ("4. Complex logic", "isinstance checks, unions, optional handling"),
    ("5. Internal helpers", "Less critical but still useful for maintenance"),
    ("6. Tests", "Lowest priority - types add less value in test code"),
]

print("Priority order for adding type annotations:")
for priority, reason in priorities:
    print(f"  {priority}")
    print(f"    {reason}")

## Practical Example: Typing a Real Module

Let's apply everything we have learned to a small but realistic module
that processes user data - demonstrating gradual typing, narrowing,
TypedDict, and proper error handling.

In [ ]:
from typing import TypedDict, TypeGuard, Literal, Final, cast
from collections.abc import Callable
import json


# --- Type definitions ---
class RawUser(TypedDict):
    """Shape of user data as received from the API."""
    id: int
    name: str
    email: str
    role: str
    active: bool


Role = Literal["admin", "editor", "viewer"]
VALID_ROLES: Final[set[str]] = {"admin", "editor", "viewer"}


class ValidatedUser(TypedDict):
    """Shape of user data after validation."""
    id: int
    name: str
    email: str
    role: Role
    active: bool


# --- Type guard ---
def is_valid_role(role: str) -> TypeGuard[Role]:
    """Check if a string is a valid Role literal."""
    return role in VALID_ROLES


# --- Validation ---
class ValidationError(Exception):
    """Raised when user data fails validation."""
    pass


def validate_user(raw: RawUser) -> ValidatedUser:
    """Validate and narrow a raw user to a validated user."""
    if not raw["name"].strip():
        raise ValidationError(f"User {raw['id']}: name is empty")

    if "@" not in raw["email"]:
        raise ValidationError(f"User {raw['id']}: invalid email {raw['email']!r}")

    role = raw["role"]
    if not is_valid_role(role):
        raise ValidationError(f"User {raw['id']}: invalid role {role!r}")

    # After TypeGuard, mypy knows role is Role
    return {
        "id": raw["id"],
        "name": raw["name"].strip(),
        "email": raw["email"].lower(),
        "role": role,
        "active": raw["active"],
    }


# --- Processing pipeline ---
Transformer = Callable[[ValidatedUser], ValidatedUser]


def apply_pipeline(
    users: list[ValidatedUser],
    *transforms: Transformer,
) -> list[ValidatedUser]:
    """Apply a sequence of transformations to validated users."""
    result = list(users)
    for transform in transforms:
        result = [transform(u) for u in result]
    return result


def deactivate_viewers(user: ValidatedUser) -> ValidatedUser:
    """Deactivate all viewer accounts."""
    if user["role"] == "viewer":
        return {**user, "active": False}
    return user


def normalize_email(user: ValidatedUser) -> ValidatedUser:
    """Ensure email is lowercase."""
    return {**user, "email": user["email"].lower()}


# --- Main ---
raw_users: list[RawUser] = [
    {"id": 1, "name": "Alice", "email": "Alice@Example.COM", "role": "admin", "active": True},
    {"id": 2, "name": "Bob", "email": "bob@test.com", "role": "viewer", "active": True},
    {"id": 3, "name": "Carol", "email": "carol@test.com", "role": "editor", "active": True},
]

# Validate
validated: list[ValidatedUser] = []
for raw in raw_users:
    try:
        validated.append(validate_user(raw))
    except ValidationError as e:
        print(f"Skipped: {e}")

# Transform
processed = apply_pipeline(validated, normalize_email, deactivate_viewers)

print("Processed users:")
for user in processed:
    status = "active" if user["active"] else "inactive"
    print(f"  {user['name']} ({user['role']}, {status}): {user['email']}")

# Test validation error
try:
    bad_user: RawUser = {
        "id": 99, "name": "Dan", "email": "invalid",
        "role": "superadmin", "active": True,
    }
    validate_user(bad_user)
except ValidationError as e:
    print(f"\nValidation caught: {e}")

## Summary

### mypy Configuration
- Configure in `pyproject.toml` under `[tool.mypy]`
- Start lenient, increase strictness gradually with per-module overrides
- Key settings: `check_untyped_defs`, `no_implicit_optional`, `warn_return_any`

### Common Errors and Fixes
- `[assignment]`: wrong type assigned to variable - fix the type or the value
- `[union-attr]`: accessing attribute on `Optional` without None check - narrow first
- `[arg-type]`: wrong argument type - pass the correct type
- `[return]` / `[return-value]`: missing or wrong return - cover all code paths

### Type Narrowing
- `isinstance(x, T)` narrows `x` to `T` in the True branch
- `x is not None` narrows `Optional[T]` to `T`
- `assert isinstance(x, T)` narrows from the assert onward
- `TypeGuard[T]` for custom narrowing functions
- `assert_never(x)` for exhaustiveness checking

### Escape Hatches
- `cast(T, value)`: tell mypy a value is type T (no runtime check)
- `# type: ignore[code]`: suppress a specific error (document why)
- `Any`: opt out of type checking for truly dynamic code
- `reveal_type(x)`: ask mypy what type it inferred

### Best Practices
- Type public APIs first, internal helpers last, tests are optional
- Prefer `Protocol` over `Any` when you need some structure
- Use `TypedDict` for JSON-like data shapes
- Use `TypeGuard` for custom validation that narrows types
- Always include error codes in `# type: ignore` comments